# THE MATHEMATICS OF LATTICE-BASED CRYPTOGRAPHY
## 2. Learning With Errors (LWE) Problem

**Credits to:**  
Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

---

### Outline
1. LWE definition  
2. LWE parameters  
3. Decisional LWE  
4. Short-secret LWE (ss-LWE)  
5. Application: Lindner-Peikert public-key encryption scheme  


### LWE Definition

**Notation**:
- $\mathbb{Z}_q = \{0,1,2,\ldots,q-1\}$  
- $x \in_R S$ means that $x$ is selected uniformly (and independently) at random from $S$  
- All vectors are column vectors

LWE was introduced by Regev in 2005.

**Definition. Learning With Errors problem**: LWE $( m,n,q,B )$  
Let $ s \in_R \mathbb{Z}^n_q $ and $ e \in_R [-B,B]^m $ where $ B \ll q/2 $.  
Given $ A \in_R \mathbb{Z}^{m \times n}_q $ and $ b = As + e \pmod{q} \in \mathbb{Z}^m_q $, find $ s $.


### LWE Example
Let $ m = 5, n = 3, q = 31 $, and $ B = 2 $.

**LWE instance:**  
$$
A = \begin{bmatrix}
11 & 3 & 27 \\
12 & 21 & 7 \\
6 & 23 & 30 \\
5 & 6 & 2 \\
21 & 0 & 14
\end{bmatrix}, \quad b = \begin{bmatrix}
25 \\
25 \\
12 \\
29 \\
17
\end{bmatrix}
$$

**LWE challenge:** We need to find  
$$ s \in \mathbb{Z}_{31}^3 \quad \text{and} \quad e \in [-2, 2]^5 \quad \text{with} \quad As + e = b \pmod{31}. $$

In fact, there are three LWE solutions:  
$$ 
s = (2, 11, 7)^T, \quad e = (-2, 0, 2, 1, 1)^T 
$$ 

$$ 
s = (27, 13, 16)^T, \quad e = (1, -2, 1, 1, 1)^T 
$$  

$$ 
s = (30, 9, 5)^T, \quad e = (-2, -1, 2, 1, -1)^T 
$$

In general, one cannot guarantee that there is a unique LWE solution. So, the LWE parameters must be carefully selected so that the probability that an LWE instance has more than one solution is negligibly small.


In [3]:
# LWE Example Code: Verify the three solutions for the instance

from typing import List, Tuple

MOD = 31

A = [
    [11, 3, 27],
    [12, 21, 7],
    [6, 23, 30],
    [5, 6, 2],
    [21, 0, 14]
]

b = [25, 25, 12, 29, 17]

print("LWE Example: m=5, n=3, q=31, B=2\n")
print("A =")
print("[ [11, 3, 27],")
print("  [12, 21, 7],")
print("  [6, 23, 30],")
print("  [5, 6, 2],")
print("  [21, 0, 14] ]\n")

print("b =")
print("[ 25,")
print("  25,")
print("  12,")
print("  29,")
print("  17 ]\n")

print("LWE challenge: find s in Z_31^3 and e in [-2,2]^5 with As + e = b (mod 31)\n")

solutions: List[Tuple[Tuple[int, int, int], Tuple[int, int, int, int, int]]] = [
    ((2, 11, 7), (-2, 0, 2, 1, 1)),
    ((27, 13, 16), (1, -2, 1, 1, 1)),
    ((30, 9, 5), (-2, -1, 2, 1, -1)),
]

print("Solutions (3):")
for s, e in solutions:
    print(f"s = {s}^T, e = {e}^T")
print()

# Verify As + e == b (mod 31)
def mat_vec_mod(M: List[List[int]], v: Tuple[int, ...], mod: int) -> List[int]:
    res = []
    for row in M:
        val = sum((row[j] * v[j]) for j in range(len(v))) % mod
        res.append(val)
    return res

for idx, (s, e) in enumerate(solutions, 1):
    As = mat_vec_mod(A, s, MOD)
    lhs = [ (As[i] + e[i]) % MOD for i in range(len(As)) ]
    ok = lhs == b
    print(f"Verify solution {idx}: As + e (mod 31) = {lhs} -> {'OK' if ok else 'MISMATCH'}")

print("\nDone.")


LWE Example: m=5, n=3, q=31, B=2

A =
[ [11, 3, 27],
  [12, 21, 7],
  [6, 23, 30],
  [5, 6, 2],
  [21, 0, 14] ]

b =
[ 25,
  25,
  12,
  29,
  17 ]

LWE challenge: find s in Z_31^3 and e in [-2,2]^5 with As + e = b (mod 31)

Solutions (3):
s = (2, 11, 7)^T, e = (-2, 0, 2, 1, 1)^T
s = (27, 13, 16)^T, e = (1, -2, 1, 1, 1)^T
s = (30, 9, 5)^T, e = (-2, -1, 2, 1, -1)^T

Verify solution 1: As + e (mod 31) = [25, 25, 12, 29, 17] -> OK
Verify solution 2: As + e (mod 31) = [25, 25, 12, 29, 17] -> OK
Verify solution 3: As + e (mod 31) = [25, 25, 12, 29, 17] -> OK

Done.


### LWE Parameter $B$
1. If $ B = 0 $ (so $ e = 0 $), then $ As = b \pmod{q} $ can be solved efficiently.
2. If $ B = (q - 1)/2 $, then finding $ s $ is information-theoretically impossible. So, we'll henceforth assume that $ B < q/4 $.
3. (Arora-Ge) If $ B $ is asymptotically smaller than $ \sqrt{n} $, then LWE can be solved in subexponential time for sufficiently large $ m \gg n $.

### LWE Parameters $m$ and $n$
- If $ m \gg n $, then one expects that there is a unique LWE solution $(s, e)$.
- Henceforth, we'll assume that $ m \gg n $.
- Uniqueness of the LWE solution is only guaranteed if no two of the $ q^n $ spheres centred at the vectors $ As \pmod{q} $ overlap.


### Decisional LWE (DLWE)
**Definition.** $\textit{Decisional LWE problem:}$ DLWE $( m, n, q, B )$

Let $ A \in_R \mathbb{Z}^{m \times n}_q $, $ s \in_R \mathbb{Z}^n_q $, $ e \in_R [-B, B]^m $ where $ B \ll q/2 $, and $ b = As + e $.

Let $ r \in_R \mathbb{Z}^m_q $.

Let $ c = b $ with probability 1/2, and $ c = r $ with probability 1/2.

Given $(A, c)$, the problem is to decide (with success probability significantly greater than 1/2) whether $ c = b $ or $ c = r $.

---

### DLWE and LWE are Equivalent
**Claim.** DLWE and LWE are equivalent.

---

### Short-Secret LWE (ss-LWE)
**Definition.** Short-secret LWE problem: ss-LWE $( m, n, q, B )$  
Let $ s \in_R [-B, B]^n $ and $ e \in_R [-B, B]^m $ where $ B \ll q/2 $.  
Given $ A \in_R \mathbb{Z}^{m \times n}_q $ and $ b = As + e \pmod{q} \in \mathbb{Z}^m_q $, find $ s $.

**Claim.** LWE and ss-LWE are equivalent.  
More precisely, ss-LWE $( m, n, q, B ) \leq$ LWE $( m, n, q, B )$ and LWE $( m, n, q, B ) \leq $ ss-LWE $( m - n, n, q, B )$.

### PKE: Key Generation
[Lindner-Peikert 2011]

**Key generation:** Alice does:

1. Select $ s \in_R [-B, B]^n $ and $ e \in_R [-B, B]^n $
2. Select $ A \in_R \mathbb{Z}_q^{n \times n} $
3. Compute $ b = As + e \pmod{q} $
4. Alice's **public key** is $(A, b)$; her **private key** is $ s $

- Determining any information about $ s $ from $(A, b)$ is an instance of **ss-DLWE $( n, n, q, B )$**

---

### PKE: Encryption and Decryption
**Encryption**: To encrypt a message $ m \in \{0,1\} $ for Alice, Bob does:

1. Obtain an authentic copy of Alice's encryption key $(A, b)$
2. Select $ r, z \in_R [-B, B]^n $ and $ z' \in_R [-B, B] $
3. Compute $ c_1 = A^T r + z $ and $ c_2 = b^T r + z' + m\lfloor q/2 \rfloor $
4. Output $ c = (c_1, c_2) $

**Note**: $ c \in \mathbb{Z}_q^n \times \mathbb{Z}_q $

---

**Decryption**: To decrypt $ c = (c_1, c_2) $, Alice does:

1. Output $ m = \text{Round}_q(c_2 - s^T c_1) $  
   Note: Alice uses her private key $ s $

$Round_q$: For $ x \in [0, q - 1] $, define  

$$
x \textit{ mods } q = 
\begin{cases} 
x & \text{if } x \leq (q - 1)/2, \\
x - q & \text{if } x > (q - 1)/2
\end{cases}
$$ 

Then  
$$
\text{Round}_q(x) =
\begin{cases} 
0, & \text{if } -q/4 < x \textit{ mods } q < q/4, \\
1, & \text{otherwise}
\end{cases}
$$


### Toy Example: PKE (1)
**Domain parameters:** $ n = 3, q = 229, B = 2 $

**Key generation:** Alice selects:  
$$
A = \begin{bmatrix}
101 & 173 & 27 \\
192 & 121 & 7 \\
116 & 223 & 30
\end{bmatrix}, \quad s = \begin{bmatrix} 2 \\ -2 \\ 1 \end{bmatrix}, \quad e = \begin{bmatrix} 0 \\ -2 \\ 1 \end{bmatrix}
$$
and computes  
$$
 b = As + e \pmod{229} = \begin{bmatrix} 112 \\ 147 \\ 17 \end{bmatrix}
$$
Alice's **encryption key** is $(A, b)$; her **decryption key** is $ s $

---

### Toy Example: PKE (2)
**Encryption**: To encrypt the plaintext bit $ m = 1 $, Bob selects  
$$
r = \begin{bmatrix} 2 \\ -2 \\ -1 \end{bmatrix}, \quad z = \begin{bmatrix} 0 \\ 1 \\ -2 \end{bmatrix}, \quad z' = -2, \text{ and computes}
$$  
$$
 c_1 = A^T r + z \pmod{229} = \begin{bmatrix} 160 \\ 111 \\ 37 \end{bmatrix} \quad \text{and} \quad c_2 = b^T r + z' + 115m \pmod{229} = 26
$$  
The **ciphertext** is $ c = (c_1, c_2) $

**Decryption**: To decrypt $ c = (c_1, c_2) $, Alice uses her decryption key $ s $ to compute  
$$
 c_2 - s^T c_1 \pmod{229} = 120
$$  
Now, $ 120 \mod 229 = -109 $, and $ \text{Round}_{229}(-109) = 1 $  
Thus, Alice recovers the plaintext $ m = 1 $


### PKE: Decryption Works
**Question**: Does decryption work?  
i.e., does $ m = \text{Round}_q(c_2 - s^T c_1) $?  

We have $ c_2 - s^T c_1 = (b^T r + z' + m \lfloor q/2 \rfloor) - s^T (A^T r + z) $  
$$ 
= (s^T A^T + e^T) r + z' + m \lfloor q/2 \rfloor - s^T (A^T r + z) 
$$  
$$ 
= e^T r - s^T z + z' + m \lfloor q/2 \rfloor 
$$

So, decryption works iff  
$$ |e^T r - s^T z + z' \mod q| < q/4 $$

Now, suppose that $ B \leq \sqrt{q/(4(2n+1))} $

Then  
$$ 
|e^T r - s^T z + z' \mod q| \leq nB^2 + nB^2 + B \leq \frac{2nq}{4(2n+1)} + \sqrt{\frac{q}{4(2n+1)}} 
$$  
$$ 
= \frac{nq}{2(2n+1)} + \sqrt{\frac{q}{4(2n+1)}} < \frac{q}{4} 
$$
so decryption works. $\square$

---

### PKE: Security
**Claim**: The Lindner-Peikert PKE is indistinguishable against chosen-plaintext attack assuming that ss-DLWE is hard.

**Proof**: The encryption operation can be written as:  
$$
\begin{bmatrix}
c_1 \\
c_2
\end{bmatrix} = 
\begin{bmatrix}
A^T \\
b^T
\end{bmatrix} r + 
\begin{bmatrix}
z \\
z'
\end{bmatrix} + 
\begin{bmatrix}
0 \\
\left\lfloor \frac{q}{2} \right\rfloor m
\end{bmatrix}
$$

By the ss-DLWE assumption,  
$$
\begin{bmatrix}
A^T \\
b^T
\end{bmatrix}
$$ is indistinguishable from random.

Again by the ss-DLWE assumption,  
$$
\begin{bmatrix}
A^T \\
b^T
\end{bmatrix} r + 
\begin{bmatrix}
z \\
z'
\end{bmatrix} = 
\begin{bmatrix}
A^T r + z \\
b^T r + z'
\end{bmatrix}
$$ is indistinguishable from random.

Thus, from the adversary's perspective, $ c_2 $ appears to be the sum of the random element  
$$ b^T r + z' \in \mathbb{Z}_q $$
 and the plaintext $ \left\lfloor \frac{q}{2} \right\rfloor m $, so the adversary can learn nothing about $ m $.


In [4]:
# Toy PKE Example Code: KeyGen, Encrypt, Decrypt (exact values)

MOD = 229

A = [
    [101, 173, 27],
    [192, 121, 7],
    [116, 223, 30]
]

s = [2, -2, 1]
e = [0, -2, 1]

# Matrix-vector multiply mod q
def mv_mod(M, v, mod):
    return [sum(M[i][j]*v[j] for j in range(len(v))) % mod for i in range(len(M))]

# Vector add mod q
def vadd_mod(u, v, mod):
    return [(u[i] + v[i]) % mod for i in range(len(u))]

b = vadd_mod(mv_mod(A, s, MOD), e, MOD)

print("Domain parameters: n=3, q=229, B=2\n")
print("Key generation:")
print("A =")
print("[ [101, 173, 27],")
print("  [192, 121, 7],")
print("  [116, 223, 30] ]")
print("s = [ 2, -2, 1 ]^T")
print("e = [ 0, -2, 1 ]^T")
print("b = As + e mod 229 =")
print("[", b[0], ",", b[1], ",", b[2], "]^T\n")

# Encryption for m=1
m = 1
r = [2, -2, -1]
z = [0, 1, -2]
zp = -2

# c1 = A^T r + z (mod q)
# Compute A^T r: equivalently dot each column of A with r
ATr = [ (A[0][j]*r[0] + A[1][j]*r[1] + A[2][j]*r[2]) % MOD for j in range(3) ]
c1 = [ (ATr[j] + z[j]) % MOD for j in range(3) ]

# c2 = b^T r + z' + 115*m (mod q)
bTr = (b[0]*r[0] + b[1]*r[1] + b[2]*r[2]) % MOD
c2 = (bTr + zp + (MOD//2)*m) % MOD

print("Encryption (m = 1):")
print("r = [ 2, -2, -1 ]^T")
print("z = [ 0, 1, -2 ]^T")
print("z' = -2")
print("c1 = A^T r + z mod 229 =")
print("[", c1[0], ",", c1[1], ",", c1[2], "]^T")
print(f"c2 = b^T r + z' + 115*m mod 229 = {c2}\n")

print("Ciphertext: c = (c1, c2)\n")

# Decryption: compute c2 - s^T c1 mod 229
sTc1 = (s[0]*c1[0] + s[1]*c1[1] + s[2]*c1[2]) % MOD
val = (c2 - sTc1) % MOD
print(f"Decryption: c2 - s^T c1 mod 229 = {val}")

# mods function
half = (MOD - 1)//2
mods = val if val <= half else val - MOD
print(f"Now, {val} mods 229 = {mods}, and Round_229({mods}) = {1 if not (-MOD/4 < mods < MOD/4) else 0}")
print("Thus, Alice recovers the plaintext m = 1")


Domain parameters: n=3, q=229, B=2

Key generation:
A =
[ [101, 173, 27],
  [192, 121, 7],
  [116, 223, 30] ]
s = [ 2, -2, 1 ]^T
e = [ 0, -2, 1 ]^T
b = As + e mod 229 =
[ 112 , 147 , 46 ]^T

Encryption (m = 1):
r = [ 2, -2, -1 ]^T
z = [ 0, 1, -2 ]^T
z' = -2
c1 = A^T r + z mod 229 =
[ 160 , 111 , 8 ]^T
c2 = b^T r + z' + 115*m mod 229 = 225

Ciphertext: c = (c1, c2)

Decryption: c2 - s^T c1 mod 229 = 119
Now, 119 mods 229 = -110, and Round_229(-110) = 1
Thus, Alice recovers the plaintext m = 1
